<a href="https://colab.research.google.com/github/shubham-75/DSA/blob/main/SQLite_DB_DatSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1: CREATE the SQLite database;**


We need to import the sqlite3 module and create the database and tables.  You'll see this follows the syntax we have used on previous weeks.

Note that we have created the student table with a primary key that is not an INTEGER.

Is this good practice?  
What are the issues and benefits of doing this?

In [1]:
import sqlite3

#This statement creates a connection labelled as conn.  This will be used throughout to ensure the consistency for when we start to query the database tables.
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE olist_customers (
    customer_id VARCHAR(32) PRIMARY KEY,
    customer_unique_id VARCHAR(32),
    customer_zip_code_prefix INT,
    customer_city VARCHAR(255),
    customer_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_geolocation (
    geolocation_zip_code_prefix INT,
    geolocation_lat FLOAT,
    geolocation_lng FLOAT,
    geolocation_city VARCHAR(255),
    geolocation_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_order_items (
    order_id VARCHAR(32),
    order_item_id INT,
    product_id VARCHAR(32),
    seller_id VARCHAR(32),
    shipping_limit_date DATETIME,
    price FLOAT,
    freight_value FLOAT,
    PRIMARY KEY (order_id, order_item_id)
);
''')

cursor.execute('''
CREATE TABLE olist_order_payments (
    order_id VARCHAR(32),
    payment_sequential INT,
    payment_type VARCHAR(50),
    payment_installments INT,
    payment_value FLOAT,
    PRIMARY KEY (order_id, payment_sequential)
);
''')

cursor.execute('''
CREATE TABLE olist_order_reviews (
    review_id VARCHAR(32) PRIMARY KEY,
    order_id VARCHAR(32),
    review_score INT,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date DATETIME,
    review_answer_timestamp DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_orders (
    order_id VARCHAR(32) PRIMARY KEY,
    customer_id VARCHAR(32),
    order_status VARCHAR(50),
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATETIME,
    order_delivered_customer_date DATETIME,
    order_estimated_delivery_date DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_products (
    product_id VARCHAR(32) PRIMARY KEY,
    product_category_name VARCHAR(255),
    product_name_lenght FLOAT,
    product_description_lenght FLOAT,
    product_photos_qty FLOAT,
    product_weight_g FLOAT,
    product_length_cm FLOAT,
    product_height_cm FLOAT,
    product_width_cm FLOAT
);
''')

cursor.execute('''
CREATE TABLE olist_sellers (
    seller_id VARCHAR(32) PRIMARY KEY,
    seller_zip_code_prefix INT,
    seller_city VARCHAR(255),
    seller_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE product_category_translation (
    product_category_name VARCHAR(255) PRIMARY KEY,
    product_category_name_english VARCHAR(255)
);
''')

#This saves the chnages to the databae.  Up unitl this point the executed SQL statement isn't stored, changes are not immediatley saved.
conn.commit()

print("Database and tables created successfully!")


Database and tables created successfully!


**STEP 2: Check Tables Created:**

Run the command to show the database tables created and the structure.

In [2]:

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)




Table: olist_customers
  Column: customer_id, Type: VARCHAR(32), NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: customer_unique_id, Type: VARCHAR(32), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_zip_code_prefix, Type: INT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_city, Type: VARCHAR(255), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_state, Type: VARCHAR(2), NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: olist_geolocation
  Column: geolocation_zip_code_prefix, Type: INT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_lat, Type: FLOAT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_lng, Type: FLOAT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_city, Type: VARCHAR(255), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_state, Type: VARCHAR(2), NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: olist_order

**STEP 3: Upload Files:**

Run this box multiple times to upload the relevant csv files. Or drag the files across to the Files window from your desktop.

Course_Table.csv, Student_Table.csv & Grade_table.csv

In [ ]:


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


**STEP 4: Load CSV files into the database tables:**

This will populate the database tables with the data from teh csv files.  No need to write INSERT statements.

You need to make sure the correct files are loaded into the corresponding tables.

In [ ]:
import csv

def import_csv_to_table(csv_file, table_name):
    #opens the file aas read only 'r', doesn't allow the origianl csv to be changed.
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
            #? creates a placeholder for each column in the CSV file. ['?','?','?'] - Join makes it a string so it can then be inserted.
            # use of the '?' reduce risk of SQL injection
            placeholders = ', '.join(['?' for _ in row])
            #Assumes that the CSV and table have the same structure (this could be an issue) Would have to specify column names if different.
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Import data from CSV files into the relevant table - Student_Table goes into student table.  teh import_csv_to_table is the function, passing the two values across.
try:
    import_csv_to_table('olist_customers_dataset.csv', 'olist_customers')
    import_csv_to_table('olist_geolocation_dataset.csv', 'olist_geolocation')
    import_csv_to_table('olist_order_items_dataset.csv', 'olist_order_items')
    import_csv_to_table('olist_order_payments_dataset.csv', 'olist_order_payments')
    #import_csv_to_table('olist_order_reviews_dataset.csv', 'olist_order_reviews')
    import_csv_to_table('olist_orders_dataset.csv', 'olist_orders')
    import_csv_to_table('olist_products_dataset.csv', 'olist_products')
    import_csv_to_table('olist_sellers_dataset.csv', 'olist_sellers')
    import_csv_to_table('product_category_name_translation.csv', 'product_category_translation')
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback changes if an error occurred



**STEP 5: Check Data has loaded**

Query each database table and load the data into a dataframe and display the first 5 lines

In [ ]:
import pandas as pd
# Query all three tables and load into pandas DataFrames
customers_df = pd.read_sql_query("SELECT * FROM olist_customers", conn)
location_df = pd.read_sql_query("SELECT * FROM olist_geolocation", conn)
orderItems_df = pd.read_sql_query("SELECT * FROM olist_order_items", conn)

#add in the other tables


# Show the first 5 lines of each DataFrame
print("Customers Table:")
print(customers_df.head(5))
print("\nLocations Table:")
print(location_df.head(5))
print("\nOrder Itmes Table:")
print(orderItems_df.head(5))




**ONLY RUN IF YOU NEED TO DELETE THE DATA IN THE TABLES**

If you run go back to **STEP 4** and re-run from there.

In [ ]:
# only run if you need to reset the tables without deleting the databae and starting again - then re-run the box previous box.
# Delete all data from the tables
cursor.execute("PRAGMA foreign_keys = OFF")
cursor.execute("DELETE FROM olist_customers")
cursor.execute("DELETE FROM olist_geolocation")
cursor.execute("DELETE FROM olist_order_items")
cursor.execute("DELETE FROM olist_order_payments")
cursor.execute("DELETE FROM olist_order_reviews")
cursor.execute("DELETE FROM olist_orders")
cursor.execute("DELETE FROM olist_products")
cursor.execute("DELETE FROM olist_sellers")
cursor.execute("DELETE FROM product_category_translation")
cursor.execute("PRAGMA foreign_keys = ON")

# Commit the changes
conn.commit()

conn.commit()
print("Database Deleted - restart.")



**STEP 6: SQL Select statements**

Run the following statements.  Please ask yoursefl the impact of each one before running.




In [ ]:
#analyse the customer list.

In [ ]:
#show number and value of orders for each customer

In [ ]:
#where do the customers come from - location

In [ ]:
#look at which sellers are selling the most items

In [ ]:
#take customer that ahs the most orders and pull out all the individual order items - is there any consistency

In [ ]:
# How are customer paying

In [ ]:
#Which payment method generates the most income

In [ ]:
#if possible, payment methods and location - how much order value - identify the area thar spends the highest

In [ ]:
#What's the range of order, the min and the max

In [ ]:
#What's the average order spend in the categories - shwo the category name in English